In [1]:
import pandas as pd
import numpy as np
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.optimizers import Adam
from tensorflow.keras.layers import Dense, LSTM
from keras.layers import  Dropout
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

from datetime import time
import numpy as np

def calculate_mape(actual, predicted):
    """
    计算 MAPE（平均绝对百分比误差）

    参数:
    
    
    actual (array-like): 实际值数组。
    predicted (array-like): 预测值数组。

    返回:
    float: MAPE 值。
    """
    # 将输入转换为 numpy 数组
    actual = np.array(actual)
    predicted = np.array(predicted)

    # 避免除以零的情况
    if np.any(actual == 0):
        raise ValueError("实际值中包含零，无法计算 MAPE。")

    # 计算 MAPE
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    return mape

2025-05-26 08:06:32.752683: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-26 08:06:32.756108: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-26 08:06:32.766612: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748246792.784008     581 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748246792.789285     581 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748246792.802313     581 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
data = pd.read_excel('CS South WFM STAT 2025_05_14.xlsx')
pd.set_option('display.max_rows', 100)
data.columns = ['conversation_start_interval_tmst', 'Time', 'offered', 'actans',
       'actabn', 'absActHt', 'absActSa', 'parent', 'child', 'fiscalDate',
       'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']

data = data[~data .offered.isna()]

data = data[data .offered != 0]
# Month one and half need to set up 
data = data[data['conversation_start_interval_tmst']<=pd.to_datetime('{} 00:00:00'.format('2025-01-15'))]
holiday_data = pd.read_excel(r'holiday.xlsx')

In [15]:
data['conversation_start_interval_tmst'].max()

Timestamp('2025-01-14 18:30:00')

In [16]:
for month_number in ['2024-11-15']:

#for month_number in ['2024-11-15','2024-12-15','2025-1-15','2025-2-15','2025-3-15']:
    #ETL Process
    data = pd.read_excel('CS South WFM STAT 2025_05_14.xlsx')
    pd.set_option('display.max_rows', 100)
    data.columns = ['conversation_start_interval_tmst', 'Time', 'offered', 'actans',
           'actabn', 'absActHt', 'absActSa', 'parent', 'child', 'fiscalDate',
           'fiscalYear', 'ficalQuarter', 'fiscalMonth', 'fiscalWeek', 'DOW']
    
    data = data[~data .offered.isna()]
    
    data = data[data .offered != -1]
    # Month one and half need to set up 
    data = data[data.conversation_start_interval_tmst<=pd.to_datetime('{} 00:00:00'.format(month_number))]
    holiday_data = pd.read_excel(r'holiday.xlsx')
    holiday_data['~is_holiday'] = 0
    Holiday_name = ['Christmas Day', 'Columbus Day',
           'Independence Day', 'Labor Day', 'Martin Luther King Jr. Day',
           'Memorial Day', "New Year's Day", "Presidents' Day", 'Thanksgiving Day',
           'Veterans Day']
    holiday_data['date'] = holiday_data.Date.dt.date
    data['datetime'] = pd.to_datetime(data.conversation_start_interval_tmst)
    data['date'] = data.datetime.dt.date
    
    data.datetime = data.datetime.dt.floor('T')
    data['datetime'] = data['datetime'].apply(
        lambda x: x.ceil('30T') if x.minute == 29 else x
    )
    data['datetime'] = data['datetime'].apply(
        lambda x: x.ceil('H') if x.minute == 59 else x
    )
    data['time'] = data.datetime.dt.time
    data = data.sort_values('datetime').reset_index(drop = True)
    data = data.sort_values(by='datetime')
    
    # 获取所有唯一的时间点（如每天的09:00, 09:30等）
    unique_times = data['time'].unique()
    
    # 获取所有唯一的日期
    unique_dates = data['date'].unique()
    data = pd.merge(data,holiday_data,on = 'date',how = 'left').fillna(1)
    data = data[data['~is_holiday'] == 1]
    
    holiday_datetime = holiday_data.date.to_numpy()
    
    start_time = 35+23 # 58 how many days in advance
    end_time = start_time+36 #36 is the train length
    # set up the interval need to forecast here
    unique_times = [ 
    time(9, 0), time(9, 30), time(10, 0),
    time(10, 30), time(11, 0), time(11, 30),
    time(12, 0), time(12, 30), time(13, 0),
    time(13, 30), time(14, 0), time(14, 30),
    time(15, 0), time(15, 30), time(16, 0),
    time(16, 30), time(17, 0), time(17, 30)
    ]
    
    add_column = ['actans','actabn','absActHt','absActSa']
    
    add_columns =  [f'actans_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'actabn_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'absActHt_{i}' for i in range(start_time+1,end_time+1) ] +\
                    [f'absActSa_{i}' for i in range(start_time+1,end_time+1) ]
    
    matrix = []
    
    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max(), freq='B')  # 仅工作日
        #full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max()+ pd.offsets.BDay(start_time), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        time_data1 = data[data['time'] == time1].set_index('date')['offered']    
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
        
        time_data2 = data[data['time'] == time2].set_index('date')['offered']    
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        year_data = time_data.shift(251)
        
    
        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
        
        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)
        
        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])
        #combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])[-start_time:]
        
        # 将结果存入矩阵
        matrix.append(combined_data_with_datetime)
    
    max_rows = max(arr.shape[0] for arr in matrix)
    
    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix)):
        if matrix[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix[i] = np.vstack([matrix[i], padding])
    
    # 将矩阵堆叠成一个大的二维数组
    matrix = np.vstack(matrix)
    
    matrix = pd.DataFrame(matrix,columns = ['datetime','offered'] +add_columns +  [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + \
                          [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)
    
    matrix['year'] = matrix.datetime.map(lambda x:x.year)
    matrix['month'] = matrix.datetime.map(lambda x:x.month)
    matrix['day'] = matrix.datetime.map(lambda x:x.day)
    matrix['hour'] = matrix.datetime.map(lambda x:x.hour)
    matrix['minute'] = matrix.datetime.map(lambda x:x.minute)
    
    matrix = matrix.sort_values('datetime').reset_index(drop = True)
    
    read_data = matrix.iloc[:]
    read_data = read_data[~read_data.offered.isna()].reset_index(drop = True)
    
    fill_na = read_data.iloc[:,1:].apply(lambda x:pd.to_numeric(x))
    
    
    
    fill_na = fill_na.interpolate(method='linear')
    
    fill_na['datetime'] = read_data.datetime
    fill_na['date'] = fill_na.datetime.dt.date
    fill_na = fill_na.dropna().reset_index(drop = True)
    
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['year'],prefix = 'year').astype(int)],axis = 1)
    fill_na['week_of_year'] = fill_na['datetime'].dt.isocalendar().week
    fill_na['weekday'] = fill_na['datetime'].dt.day_name()
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na = pd.concat([fill_na,pd.get_dummies(fill_na['weekday'],prefix = 'weekday').astype(int)],axis = 1)
    
    matrix_predict = []
    
    # 遍历每个时间点
    for time,time1,time2 in zip(unique_times,np.roll(unique_times, shift=-1),np.roll(unique_times, shift=1)):
        # 过滤出当前时间点的数据
        time_data = data[data['time'] == time].set_index('date')['offered']    
        full_dates = pd.date_range(start=unique_dates.min(), end=unique_dates.max()+ pd.offsets.BDay(start_time), freq='B')  # 仅工作日
        full_dates = full_dates.difference(holiday_datetime)
        time_data = time_data.groupby(time_data.index).sum()    
        time_data = time_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data = pd.concat([time_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        add_data = data[data['time'] == time].set_index('date')[add_column]
        add_data = add_data.groupby(add_data.index).sum()
        add_data = add_data.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        add_data_ = pd.concat([add_data.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        
        time_data1 = data[data['time'] == time1].set_index('date')['offered'] 
        time_data1 = time_data1.groupby(time_data1.index).sum()
        time_data1 = time_data1.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data1 = pd.concat([time_data1.shift(i) for i in range(start_time,end_time)], axis=1)
        
        time_data2 = data[data['time'] == time2].set_index('date')['offered']  
        time_data2 = time_data2.groupby(time_data2.index).sum()
        time_data2 = time_data2.reindex(full_dates)  # 重新索引，缺失数据用NaN填充    
        shifted_data2 = pd.concat([time_data2.shift(i) for i in range(start_time,end_time)], axis=1)
        
        
        year_data = time_data.shift(251)
        
    
        # 将当前时间点、前半小时和后半小时的数据拼接
        combined_data = pd.concat([time_data,add_data_, shifted_data,shifted_data1,shifted_data2,year_data], axis=1)
        
        # 创建日期+时间列
        datetime_column = np.array([pd.Timestamp(date) + pd.Timedelta(hours=time.hour, minutes=time.minute) for date in full_dates]).reshape(-1, 1)
        
        # 将日期+时间列添加到数据中
        combined_data_with_datetime = np.hstack([datetime_column,combined_data.to_numpy()])[-start_time:]
        
        
        
        # 将结果存入矩阵
        matrix_predict.append(combined_data_with_datetime)
    
    max_rows = max(arr.shape[0] for arr in matrix_predict)
    # 将每个时间点的数据填充到最大行数
    for i in range(len(matrix_predict)):
        if matrix_predict[i].shape[0] < max_rows:
            padding = np.full((max_rows - matrix_predict[i].shape[0], 11), np.nan)  # 用NaN填充（10列数据 + 1列时间）
            matrix_predict[i] = np.vstack([matrix_predict[i], padding])
    
    # 将矩阵堆叠成一个大的二维数组
    matrix_predict = np.vstack(matrix_predict)
    
    matrix_predict = pd.DataFrame(matrix_predict,columns = ['datetime','offered'] + add_columns+ [f'freq_{i}' for i in range(start_time+1,end_time+1) ]\
                          + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_next{i}' for i in range(start_time+1,end_time+1) ]+ \
                           ['years_data']).sort_values('datetime').reset_index(drop = True)
    
    matrix_predict['year'] = matrix_predict.datetime.map(lambda x:x.year)
    matrix_predict['month'] = matrix_predict.datetime.map(lambda x:x.month)
    matrix_predict['day'] = matrix_predict.datetime.map(lambda x:x.day)
    matrix_predict['hour'] = matrix_predict.datetime.map(lambda x:x.hour)
    matrix_predict['minute'] = matrix_predict.datetime.map(lambda x:x.minute)
    
    matrix_predict = matrix_predict.sort_values('datetime').reset_index(drop = True)
    
    read_data_predict = matrix_predict.iloc[:]
    
    
    
    
    fill_na_predict = read_data_predict.iloc[:,2:].apply(lambda x:pd.to_numeric(x))
    
    fill_na_predict = fill_na_predict.interpolate(method='linear')
    fill_na_predict = fill_na_predict.fillna(1)
    
    fill_na_predict['datetime'] = read_data_predict.datetime
    fill_na_predict['date'] = fill_na_predict.datetime.dt.date
    
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['month'],prefix = 'month').astype(int)],axis = 1)
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['year'], prefix = 'year').astype(int)],axis = 1)
    fill_na_predict['week_of_year'] = fill_na_predict['datetime'].dt.isocalendar().week
    fill_na_predict['weekday'] = fill_na_predict['datetime'].dt.day_name()
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['week_of_year'],prefix = 'week_of_year').astype(int)],axis = 1)
    fill_na_predict = pd.concat([fill_na_predict,pd.get_dummies(fill_na_predict['weekday'], prefix = 'weekday').astype(int)],axis = 1)
    
    fill_na_predict
    
    X_columns = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
                [f'year_{i}' for i in np.unique(fill_na.year)] +\
                [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                    add_columns
                
    X_columns2 = [f'freq_{i}' for i in range(start_time+1,end_time+1) ] + [f'freq_last{i}' for i in range(start_time+1,end_time+1) ] +\
                [f'freq_next{i}' for i in range(start_time+1,end_time+1) ] + ['years_data'] + [f'month_{i}' for i in range(1,13)] +\
                [f'year_{i}' for i in np.unique(fill_na.year)] +\
                [f'week_of_year_{i}' for i in fill_na.week_of_year.drop_duplicates().tolist()] +\
                    add_columns
    
    
    lstm_data = fill_na.dropna(axis = 0)
    
    lstm_data
    
    for i in X_columns:
        try:
            fill_na_predict[i]
        except:
            fill_na_predict[i] = 0
    
    lstm_data = lstm_data[lstm_data.year>=2021].reset_index(drop = True)
    
    week_dict = {'Friday':5, 'Monday':1, 'Tuesday':2, 'Wednesday':3, 'Thursday':4}
    
    lstm_data['week_num'] = lstm_data['weekday'].map(lambda x:week_dict[x])
    fill_na_predict['week_num'] = fill_na_predict['weekday'].map(lambda x:week_dict[x])

    
    all_predict = pd.DataFrame()
    all_feature = pd.DataFrame()
    #for week_ in [1,2,3,4,5]:
    for (hour,minute) in lstm_data[['hour','minute']].drop_duplicates().sort_values(['hour','minute']).to_numpy()[:]:
        now_lstm_data = lstm_data[(lstm_data.hour == hour)&(lstm_data.minute == minute)]
        #now_lstm_data = now_lstm_data[now_lstm_data.week_num == week_]
        now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
        #now_fill_na_predict = now_fill_na_predict[now_fill_na_predict.week_num == week_]
    
        now_time_list = now_lstm_data.datetime.to_numpy()
        now_lstm_data = now_lstm_data[['offered']+['datetime'] +X_columns].reset_index(drop = True)
        # 归一化数据
    
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-start_time])
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集
        X_train_1, y_train =now_lstm_data[X_columns][:-start_time] , scaled_data
    
        X_test_1, y_test =now_lstm_data[X_columns][-start_time:] , now_lstm_data[['offered']][-start_time:]
    
        X_train_1 = np.array(X_train_1)
        y_train = np.array(y_train)
        X_test_1 = np.array(X_test_1)
        y_test = np.array(y_test)
        X_test_1 = X_test_1.reshape(X_test_1.shape[0], X_test_1.shape[1], 1)
    
        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_1 = X_train_1.reshape(X_train_1.shape[0], X_train_1.shape[1], 1)
        # 构建LSTM模型
        model_1 = Sequential()
        model_1.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_1.add(LSTM(50, return_sequences=False))
        model_1.add(Dense(25))
        model_1.add(Dense(1))
    
        # 编译模型
        model_1.compile(optimizer='adam', loss='mean_squared_error')
        model_1.fit(X_train_1, y_train, batch_size=32, epochs=20)
        train_predict_1 = model_1.predict(X_train_1)
        train_predict_1 = scaler.inverse_transform(train_predict_1)
        True_data_1 = scaler.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_1, True_data_1))
        return_y_1 =  model_1.predict(X_test_1)
        return_y_pred_1 = scaler.inverse_transform(np.array(return_y_1).reshape(-1, 1))
        now_predict_1 = pd.DataFrame(return_y_pred_1,columns = ['predict'])
        now_predict_1['datetime'] = now_time_list[-start_time:]
        now_lstm_data = now_lstm_data[['offered'] +X_columns2].reset_index(drop = True)
        scaler = MinMaxScaler(feature_range=(0, 1))
        scaled_data = scaler.fit_transform(now_lstm_data[['offered']][:-start_time])
        # 设置时间步长
        time_step = start_time
        # 创建训练数据集
    
        X_train_2, y_train =now_lstm_data[X_columns2][:-start_time] , scaled_data
    
        X_test_2, y_test =now_lstm_data[X_columns2][-start_time:] , now_lstm_data[['offered']][-start_time:]
    
        X_train_2 = np.array(X_train_2)
        y_train = np.array(y_train)
        X_test_2 = np.array(X_test_2)
        y_test = np.array(y_test)
        X_test_2 = X_test_2.reshape(X_test_2.shape[0], X_test_2.shape[1], 1)
    
        # 重塑输入数据为 [samples, time steps, features] 格式
        X_train_2 = X_train_2.reshape(X_train_2.shape[0], X_train_2.shape[1], 1)
        # 构建LSTM模型
        model_2 = Sequential()
        model_2.add(LSTM(50, return_sequences=True, input_shape=(time_step, 1)))
        model_2.add(LSTM(50, return_sequences=False))
        model_2.add(Dense(25))
        model_2.add(Dense(1))
    
        # 编译模型
        model_2.compile(optimizer='adam', loss='mean_squared_error')
        model_2.fit(X_train_2, y_train, batch_size=32, epochs=20)
        train_predict_2 = model_2.predict(X_train_2)
        train_predict_2 = scaler.inverse_transform(train_predict_2)
        True_data_2 = scaler.inverse_transform(y_train.reshape(-1, 1))
        print(calculate_mape(train_predict_2, True_data_2))
        return_y_2 =  model_2.predict(X_test_2)
        return_y_pred_2 = scaler.inverse_transform(np.array(return_y_2).reshape(-1, 1))
        now_predict_2 = pd.DataFrame(return_y_pred_2,columns = ['predict'])
        now_predict_2['datetime'] = now_time_list[-start_time:]
        print(calculate_mape(y_test, return_y_pred_2))
        if calculate_mape(y_test, return_y_pred_1) < calculate_mape(y_test, return_y_pred_2):
            all_predict = pd.concat([now_predict_1,all_predict])
            future_X = now_fill_na_predict[X_columns].to_numpy()
            future_data = scaler.inverse_transform(np.array(\
                            model_1.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            all_feature = pd.concat([all_feature,future_data])
        else:
            all_predict = pd.concat([now_predict_2,all_predict])
            future_X = now_fill_na_predict[X_columns2].to_numpy()
            future_data = scaler.inverse_transform(np.array(\
                            model_2.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
            future_data = pd.DataFrame(future_data,columns = ['predict'])
            all_feature = pd.concat([all_feature,future_data])
        #print('Interval Hour:{}, Minute:{} has been trained'.format(hour,minute)
    all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
    all_predict = all_predict.sort_values('datetime').reset_index(drop = True)
    all_feature.to_excel(r'return_month_{}.xlsx'.format(month_number))

/tmp/ipykernel_224728/4027003370.py:24: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  data.datetime = data.datetime.dt.floor('T')


Epoch 1/20


E0000 00:00:1747145764.140436  224728 cuda_executor.cc:1228] INTERNAL: CUDA Runtime error: Failed call to cudaGetRuntimeVersion: Error loading CUDA libraries. GPU will not be used.: Error loading CUDA libraries. GPU will not be used.
W0000 00:00:1747145764.166294  224728 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


10/10 ━━━━━━━━━━━━━━━━━━━━ 4s 227ms/step - loss: 0.2456
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 226ms/step - loss: 0.0333
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 229ms/step - loss: 0.0277
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 228ms/step - loss: 0.0250
Epoch 5/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 230ms/step - loss: 0.0241
Epoch 6/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 223ms/step - loss: 0.0240
Epoch 7/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 208ms/step - loss: 0.0252
Epoch 8/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 232ms/step - loss: 0.0267
Epoch 9/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 231ms/step - loss: 0.0240
Epoch 10/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 220ms/step - loss: 0.0241
Epoch 11/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 222ms/step - loss: 0.0251
Epoch 12/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 225ms/step - loss: 0.0246
Epoch 13/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 229ms/step - loss: 0.0237
Epoch 14/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 221ms/step - loss: 0.0247
Epoch 15/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 219ms/step - loss: 0.0236


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


...

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 203ms/step - loss: 0.0388
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 223ms/step - loss: 0.0217
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 218ms/step - loss: 0.0198
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 230ms/step - loss: 0.0193
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 215ms/step - loss: 0.0191
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 216ms/step - loss: 0.0185
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 194ms/step - loss: 0.0188
Epoch 8/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 208ms/step - loss: 0.0172
Epoch 9/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 223ms/step - loss: 0.0161
Epoch 10/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 225ms/step - loss: 0.0166
Epoch 11/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 195ms/step - loss: 0.0166
Epoch 12/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 176ms/step - loss: 0.0167
Epoch 13/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 219ms/step - loss: 0.0172
Epoch 14/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 213ms/step - loss: 0.0166
Epoch 15/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 221ms/step - loss: 0.0155


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 219ms/step - loss: 0.2716
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 195ms/step - loss: 0.0327
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 201ms/step - loss: 0.0247
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 214ms/step - loss: 0.0216
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 208ms/step - loss: 0.0192
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 184ms/step - loss: 0.0159
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 208ms/step - loss: 0.0179
Epoch 8/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 213ms/step - loss: 0.0182
Epoch 9/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 211ms/step - loss: 0.0210
Epoch 10/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 220ms/step - loss: 0.0159
Epoch 11/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 195ms/step - loss: 0.0175
Epoch 12/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 212ms/step - loss: 0.0171
Epoch 13/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 225ms/step - loss: 0.0156
Epoch 14/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step - loss: 0.0172
Epoch 15/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 220ms/step - loss: 0.0186


KeyboardInterrupt: 

In [67]:
all_predict['diff'] = (all_predict.predict - all_predict.offered)

all_predict['hour'] = all_predict.datetime.dt.hour

pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])

calculate_mape(all_predict.offered,all_predict.predict)

all_predict.to_excel(r'return{}.xlsx'.format(month_number))

AttributeError: 'DataFrame' object has no attribute 'offered'

In [68]:
all_predict

,predict,datetime,offered_x,diff,hour,offered_y
0,55.317352,2024-03-19 17:30:00,NaN,NaN,NaN,52.0
1,57.205730,2024-03-20 17:30:00,NaN,NaN,NaN,50.0
2,57.433464,2024-03-21 17:30:00,NaN,NaN,NaN,75.0
3,56.991905,2024-03-22 17:30:00,NaN,NaN,NaN,36.0
4,56.519444,2024-03-25 17:30:00,NaN,NaN,NaN,81.0
...,...,...,...,...,...,...
1097,151.073898,2024-09-19 11:00:00,1.0,150.073898,11.0,1.0
1098,152.045944,2024-09-19 13:00:00,2.0,150.045944,13.0,2.0
1099,130.966293,2024-09-19 16:00:00,2.0,128.966293,16.0,2.0
1100,85.822189,2024-09-19 16:30:00,2.0,83.822189,16.0,2.0


In [57]:
all_predict = pd.concat([now_predict_1,all_predict])
future_X = now_fill_na_predict[X_columns].to_numpy()
future_data = scaler.inverse_transform(np.array(\
                model_1.predict(future_X.reshape(future_X.shape[0], future_X.shape[1], 1))).reshape(-1, 1))
future_data = pd.DataFrame(future_data,columns = ['predict'])
all_feature = pd.concat([all_feature,future_data])

2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step


In [58]:
all_predict

,predict,datetime,offered,diff,hour
0,55.317352,2024-03-19 17:30:00,NaN,NaN,NaN
1,57.205730,2024-03-20 17:30:00,NaN,NaN,NaN
2,57.433464,2024-03-21 17:30:00,NaN,NaN,NaN
3,56.991905,2024-03-22 17:30:00,NaN,NaN,NaN
4,56.519444,2024-03-25 17:30:00,NaN,NaN,NaN
...,...,...,...,...,...
1039,151.073898,2024-09-19 11:00:00,1.0,150.073898,11.0
1040,152.045944,2024-09-19 13:00:00,2.0,150.045944,13.0
1041,130.966293,2024-09-19 16:00:00,2.0,128.966293,16.0
1042,85.822189,2024-09-19 16:30:00,2.0,83.822189,16.0


In [59]:
all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])

In [60]:
all_predict.sort_values('datetime')

,predict,datetime,offered_x,diff,hour,offered_y
0,55.317352,2024-03-19 17:30:00,NaN,NaN,NaN,52.0
58,55.317352,2024-03-19 17:30:00,52.0,3.317352,17.0,52.0
59,134.345093,2024-03-20 09:00:00,101.0,33.345093,9.0,101.0
1,57.205730,2024-03-20 17:30:00,NaN,NaN,NaN,50.0
60,57.205730,2024-03-20 17:30:00,50.0,7.205730,17.0,50.0
...,...,...,...,...,...,...
1097,151.073898,2024-09-19 11:00:00,1.0,150.073898,11.0,1.0
1098,152.045944,2024-09-19 13:00:00,2.0,150.045944,13.0,2.0
1099,130.966293,2024-09-19 16:00:00,2.0,128.966293,16.0,2.0
1100,85.822189,2024-09-19 16:30:00,2.0,83.822189,16.0,2.0


In [61]:
all_feature = all_feature[:-58]

In [62]:
all_feature

,predict
0,123.481262
1,123.719040
2,117.245575
3,104.785278
4,74.003563
...,...
53,15.125546
54,-16.117435
55,1.071325
56,-15.810401


In [63]:
time_ = []
for (hour,minute) in fill_na_predict[['hour','minute']].drop_duplicates().to_numpy()[:]:
    now_fill_na_predict = fill_na_predict[(fill_na_predict.hour == hour)&(fill_na_predict.minute == minute)]
    time_.append(now_fill_na_predict.datetime.to_numpy())
all_feature['datetime'] =  np.hstack(time_)
all_feature.predict = all_feature.predict.map(lambda x:max(0,x))
all_feature.to_excel(r'feature2.xlsx')

In [64]:
all_predict

,predict,datetime,offered_x,diff,hour,offered_y
0,55.317352,2024-03-19 17:30:00,NaN,NaN,NaN,52.0
1,57.205730,2024-03-20 17:30:00,NaN,NaN,NaN,50.0
2,57.433464,2024-03-21 17:30:00,NaN,NaN,NaN,75.0
3,56.991905,2024-03-22 17:30:00,NaN,NaN,NaN,36.0
4,56.519444,2024-03-25 17:30:00,NaN,NaN,NaN,81.0
...,...,...,...,...,...,...
1097,151.073898,2024-09-19 11:00:00,1.0,150.073898,11.0,1.0
1098,152.045944,2024-09-19 13:00:00,2.0,150.045944,13.0,2.0
1099,130.966293,2024-09-19 16:00:00,2.0,128.966293,16.0,2.0
1100,85.822189,2024-09-19 16:30:00,2.0,83.822189,16.0,2.0


In [65]:
all_feature

,predict,datetime
0,123.481262,2024-09-17 09:00:00
1,123.719040,2024-09-18 09:00:00
2,117.245575,2024-09-19 09:00:00
3,104.785278,2024-09-20 09:00:00
4,74.003563,2024-09-23 09:00:00
...,...,...
53,15.125546,2024-12-04 17:30:00
54,0.000000,2024-12-05 17:30:00
55,1.071325,2024-12-06 17:30:00
56,0.000000,2024-12-09 17:30:00


In [66]:
calculate_mape(all_predict.predict, all_predict.offered)

AttributeError: 'DataFrame' object has no attribute 'offered'

In [ ]:
(np.mean((all_predict.predict-all_predict.offered)**2))**0.5

In [ ]:
all_predict['hour'] = all_predict.datetime.dt.hour

In [42]:
for hour in np.unique(all_predict.hour):
    _ = all_predict[all_predict.hour == hour]
    print(f'hour is {hour} the mape ：',calculate_mape(_.predict, _.offered))

AttributeError: 'DataFrame' object has no attribute 'offered'

In [ ]:
for hour in np.unique(all_predict.hour):
    _ = all_predict[all_predict.hour == hour]
    print(f'hour is {hour} the mse ：',(np.mean((_.predict-_.offered)**2))**0.5)

In [ ]:
_ = all_predict[all_predict.hour == 11]

In [ ]:
calculate_mape(_.predict, _.offered)

In [ ]:
all_predict

In [ ]:
feature_[['predict','datetime']]

In [ ]:
all_predict = pd.merge(all_predict,lstm_data[['offered','datetime']],on = ['datetime'])
all_predict = all_predict.sort_values('datetime').reset_index(drop = True)


In [ ]:
all_predict.to_excel(r'return.xlsx')

In [ ]:
datetime

In [ ]:
pd.set_option('display.max_rows', None)

In [ ]:
data11 = lstm_data[lstm_data.hour == 11]

In [ ]:
data11.groupby('datetime').min().offered.sort_values()

In [ ]:

import plotly.graph_objects as go
import numpy as np
import matplotlib.pyplot as plt

# 创建示例数据
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=all_predict['datetime'],  # 将 '日期' 列作为 X 轴
    y=all_predict['offered'],  # 将 '销售额' 列作为 Y 轴
    mode='lines+markers',  # 折线 + 数据点
    name='真实值'  # 图例名称
))
fig.add_trace(go.Scatter(
    x=all_predict['datetime'],  # 将 '日期' 列作为 X 轴
    y=all_predict['predict'],  # 将 '销售额' 列作为 Y 轴
    mode='lines+markers',  # 折线 + 数据点
    name='预测值'  # 图例名称
))

# 设置坐标轴名称
fig.update_layout(
    title='折线图示例',
    xaxis_title='X轴',
    yaxis_title='Y轴',
    template='plotly_white'
)
fig.show()

In [ ]:

# 可视化训练数据的预测结果
plt.figure(figsize=(10, 6))
plt.plot( all_predict['datetime'], all_predict['offered'] , label='Actual Data')
plt.plot( all_predict['datetime'], all_predict['predict'], label='Predicted Data')
plt.legend()
plt.show()
# 预测未来数据
future_steps = 30
future_predictions = []

last_sequence = X_train[-1]


In [ ]:
now_lstm_data

In [ ]:
now_lstm_data.offered.min()

In [ ]:
_ = lstm_data.groupby(['year','month','day']).offered.sum()
_.sort_values()[:30]

In [ ]:
now_lstm_data[X_columns]

In [ ]:

# 可视化训练数据的预测结果
plt.figure(figsize=(10, 6))
plt.plot(now_lstm_data[['offered']][1000:].to_numpy(), label='Actual Data')
plt.plot(return_y_pred, label='Predicted Data')
plt.legend()
plt.show()
# 预测未来数据
future_steps = 30
future_predictions = []

last_sequence = X_train[-1]


In [ ]:
predict_y =  model.predict( now_lstm_data[X_columns][:1000].to_numpy().reshape(-1, X_train.shape[1], 1))
predict_y = scaler.inverse_transform(np.array(predict_y).reshape(-1, 1))

In [ ]:
calculate_mape(now_lstm_data[['offered']][:1000].to_numpy(), predict_y)

In [ ]:
_ = ((np.abs((now_lstm_data[['offered']][1000:].to_numpy() - return_y_pred)/now_lstm_data[['offered']][1000:].to_numpy())))*100

In [ ]:
_.sort()

In [ ]:
np.argmax(_)

In [ ]:
_[298]

In [ ]:
now_lstm_data[['offered']][1000:].to_numpy()[298]